In [ ]:
import fuzzylite as fl
import numpy as np

import plotly
import plotly.express as px
import plotly.graph_objects as go

from pathlib import Path

from IPython.display import display, HTML, SVG

plotly.offline.init_notebook_mode()
display(
    HTML(
        '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)

In [ ]:
x = np.linspace(-1, 1, 10_000)
# x = fl.Op.midpoints(-0.5, 0.5, 1_000)
x_min = np.min(x)
x_mean = np.mean(x)
x_max = np.max(x)
x_diff = x_max - x_min

x_min, x_mean, x_max, x_diff

In [ ]:
def show(term: fl.Term) -> SVG:
    width = 10
    colors = ["blue", "red"]
    if not term.name:
        term.name = fl.Op.class_name(term)
    if isinstance(term, fl.Discrete):
        fig = px.scatter(
            x=term.x(),
            y=term.y(),
        ).update_traces(marker=dict(size=width, color=colors[0]))
    elif isinstance(term, fl.Aggregated):
        fig = go.Figure()
        for i, term_i in enumerate(term.terms):
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=term_i.membership(x),
                    mode="lines",
                    showlegend=False,
                    line=go.scatter.Line(color=colors[(i + 1) % len(colors)], width=width),
                )
            )
    elif isinstance(term, (fl.Constant, fl.Linear, fl.Function)):
        names = {
            fl.Constant: "\mu(x)=k",
            fl.Linear: "\mu(x)=c_iv_i+k",
            fl.Function: "\mu(x)=f(x)",
        }
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=[x_mean],
                y=[0.8],
                mode="text",
                text=rf"$\Huge {names[type(term)]}$",
                textposition="middle center",
            )
        )
        fig.update_layout(yaxis_range=[0, 1])
    else:
        fig = px.line(
            x=x,
            y=term.membership(x),
        ).update_traces(line=dict(width=width, color=colors[0]))

    fig.update_layout(
        autosize=False,
        #         yaxis_title=None,
        #         xaxis_title=None,
        width=640,
        height=480,
        yaxis_title=r"$\Large\mu(x)$",
        xaxis_title="$\Large x$",
        xaxis_range=[x_min, x_max],
        title=None,  # term.name,
        title_font=dict(size=30),
        title_x=0.5,
        title_y=0.5,
    )
    #     fig.update_yaxes(autorangeoptions_include=[0, 1], showticklabels=False)
    #     fig.update_xaxes(autorangeoptions_include=[x_min, x_max], showticklabels=False)
    fig.update_yaxes(autorangeoptions_include=[0, 1])
    fig.update_xaxes(autorangeoptions_include=[x_min, x_max])
    return SVG(fig.to_image(format="svg"))

In [ ]:
terms = [
    # Basic
    fl.Triangle("", x_min, x_mean, x_max),
    fl.Trapezoid("", x_min, x_min * 0.25, x_max * 0.25, x_max),
    fl.Rectangle("", 0.5 * x_min, 0.50 * x_max),
    fl.SemiEllipse("", 0.75 * x_min, 0.75 * x_max),
    fl.SemiEllipse("", 0.5 * x_min, 0.5 * x_max).discretize(x_min, x_max, 50),
    # Extended
    fl.Cosine("", x_mean, x_diff),
    fl.Gaussian("", x_mean, 0.25 * x_diff),
    fl.Bell("", x_mean, 0.25 * x_diff, 3.0),
    fl.PiShape("", x_min, x_mean, x_mean, x_max),
    fl.SigmoidDifference("", x_min * 0.75, 10 / x_diff, 10 / x_diff, x_max * 0.75),
    fl.SigmoidProduct("", x_min * 0.75, 10 / x_diff, -10 / x_diff, x_max * 0.75),
    fl.Spike("", x_mean, 3 * x_diff),
    fl.GaussianProduct("", x_mean - 0.5, 0.1 * x_diff, x_mean + 0.5, 0.1 * x_diff),
    # Edge
    fl.Aggregated(
        "Binary",
        aggregation=fl.UnboundedSum(),
        terms=[
            fl.Binary("", 0.5 * x_min, -fl.inf),
            fl.Binary("", 0.5 * x_max, fl.inf),
        ],
    ),
    # TODO:
    #     fl.Aggregated(
    #         "Concave",
    #         aggregation=fl.Maximum(),
    #         terms=[
    #             fl.Concave("", x_min * 0.5, x_min * 0.5),
    #             fl.Concave("", x_max * 0.5, x_max * 0.5),
    #         ],
    #     ),
    fl.Aggregated(
        "Concave",
        aggregation=fl.Maximum(),
        terms=[
            fl.Concave("", x_min * 0.25, x_min * 0.5),
            fl.Concave("", x_max * 0.25, x_max * 0.5),
        ],
    ),
    fl.Aggregated(
        "Ramp",
        aggregation=fl.Maximum(),
        terms=[
            fl.Ramp("", x_min * 0.25, x_min * 0.5),
            fl.Ramp("", x_max * 0.25, x_max * 0.5),
        ],
    ),
    fl.Aggregated(
        "Sigmoid",
        aggregation=fl.Maximum(),
        terms=[
            fl.Sigmoid("", x_min * 0.4, -20 / x_diff),
            fl.Sigmoid("", x_max * 0.4, 20 / x_diff),
        ],
    ),
    fl.Aggregated(
        "ZShape - SShape",
        aggregation=fl.Maximum(),
        terms=[
            fl.ZShape("", x_min, x_mean),
            fl.SShape("", x_mean, x_max),
        ],
    ),
    fl.ZShape("", x_min, x_mean),
    fl.SShape("", x_mean, x_max),
    fl.Aggregated(
        "Arc",
        aggregation=fl.UnboundedSum(),
        terms=[
            fl.Arc("", x_min * 0.25, x_min),
            fl.Arc("", x_max * 0.25, x_max),
        ],
    ),
    # Function
    fl.Constant("", 100),
    fl.Linear("", [0.5, 1]),
    fl.Function("", "x"),
]
print(len(terms))
for term in terms:
    print(term)
    plot = show(term)
    display(plot)
    Path("../../docs/image/term/" + term.name + ".svg").write_text(plot.data)